In [87]:
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize

Parámetros

In [88]:
y = 10
a = 1
b = 1
beta = 0.9
T = 10

# Initial guess for optimizations
x0 = [1, 1]  # Adjust as needed

Funciones

In [89]:
# Define the pi_rho function
def pi_rho(tau, g, d, h, eta_ti):
    return 1 / (1 + np.exp(d * g - h * tau - eta_ti))

In [90]:
# Define the U function
def U(c, g, a, b):
    return a * np.log(c) + b * np.log(g)

# Define the V function
def V(k):
    return -np.log(k)

# Define the W function
def W(g, tau, epsilon, y, a, b, beta):
    return U(y - tau, g, a, b) + beta * V(tau + epsilon - g)


Funciones estocasticas

In [91]:
def generate_ar1(phi, sigma, T):
    # Initial value is a random normal variable
    series = [np.random.normal(0, sigma)]

    for t in range(1, T):
        series.append(phi * series[t-1] + np.random.normal(0, sigma))

    return np.array(series)

In [92]:
length = 15  # Length of the time series
# Parameters for the AR(1) processes
# Incumbent
phi_alpha_i = 10  # AR coefficient for alpha
sigma_alpha_i = 0.1  # Std dev for alpha

phi_q_i = 10  # AR coefficient for q
sigma_q_i = 0.1  # Std dev for q

# Oponent
phi_alpha_o = 10  # AR coefficient for alpha
sigma_alpha_o = 0.1  # Std dev for alpha

phi_q_o = 10  # AR coefficient for q
sigma_q_o = 0.1  # Std dev for q

#_________________________
# Generate the AR(1) series
# Incumbent
alpha_series_i = generate_ar1(phi_alpha_i, sigma_alpha_i, length)
q_series_i = generate_ar1(phi_q_i, sigma_q_i, length)

# Oponent
alpha_series_o = generate_ar1(phi_alpha_o, sigma_alpha_o, length)
q_series_o = generate_ar1(phi_q_o, sigma_q_o, length)

#_________________________
# Calculate epsilon and eta
# Incumbent
epsilon_ti = alpha_series_i[1:] + alpha_series_i[:-1]
eta_ti = q_series_i[1:] + q_series_i[:-1]

# Oponent
epsilon_to = alpha_series_o[1:] + alpha_series_o[:-1]
eta_to = q_series_o[1:] + q_series_o[:-1]


Estimacion bayesiana de los valores de Omega

In [93]:
# Just for the incumbent, because we can't condition, the opponent, rudimentary bayesian theory estimation
# Example prior parameters
phi_alpha_prior = 10  # AR observed coefficient for alpha
sigma_alpha_prior = 0.1  # Std dev observed coefficient for alpha

# Generate the AR(1) series
alpha_prior_series = generate_ar1(phi_alpha_prior, sigma_alpha_prior, T)

# Observed Epsilon
observed_epsilon = alpha_prior_series[1:] + alpha_prior_series[:-1]

# Update phi's prior based on observed data (this is a simplified example)
# In reality, this step can be complex and may require numerical methods
phi_mu_posterior = np.mean(observed_epsilon)
phi_sigma_posterior = np.std(observed_epsilon) / np.sqrt(len(observed_epsilon))

# Update sigma's prior similarly (this is also simplified)
# sigma_mu_posterior = np.std(observed_epsilon)
# sigma_sigma_posterior = np.std(observed_epsilon) / np.sqrt(len(observed_epsilon))

# Simulate future values of epsilon
simulated_epsilons = np.random.normal(phi_mu_posterior, phi_sigma_posterior, 15)


In [94]:
def constraint1(x, y):
    # y - tau >= 0
    return y - x[0]

def constraint2(x):
    # g >= 0
    return x[1]

def constraint3(x, epsilon):
    # tau + epsilon - g >= 0
    return x[0] + epsilon - x[1]

In [95]:
def optimizador_W(T, y, beta, a, b, epsilon):
    tau_optimo = []
    g_optimo = []
    valores_max = []

    for t in range(T):
        # Itera sobre los valores de competencia del agente
        epsilon_t = epsilon[t]

        # Restricciones por cada periodo de tiempo(epsilon)
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combinamos las restricciones en una lista
        constraints_t = [con1, con2, con3]

        # Optimiza para ese periodo de tiempo t
        resultado_t = minimize(W, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Guarda los resultados para cada periodo de tiempo
        tau_optimo.append(resultado_t.x[0])
        g_optimo.append(resultado_t.x[1])
        valores_max.append(resultado_t.fun)

    return tau_optimo, g_optimo, valores_max

In [96]:
epsilon_to

array([-8.73773280e-01, -8.92801577e+00, -8.92993282e+01, -8.93049659e+02,
       -8.93066975e+03, -8.93068241e+04, -8.93068278e+05, -8.93068259e+06,
       -8.93068258e+07, -8.93068258e+08, -8.93068258e+09, -8.93068258e+10,
       -8.93068258e+11, -8.93068258e+12])

In [97]:
simulated_epsilons

array([51451690.29278793,  8915559.67123432, 16739522.47215603,
        4167412.54001501, 17936297.77129753, 25116188.75054247,
       25819062.43682639, 17707261.54960422, 33506400.08349054,
       30999659.32455696, -2711433.65254923, 10740402.72903084,
       10410695.84614083, 33051716.12856565, 23781758.35489866])

In [75]:
# For Omega_i
Omega_i_values = []
for epsilon in simulated_epsilons:  # Optimize Omega with the epsilon conditioned observed values
    Omega_i_values.append(optimizador_W(T, y, beta, a, b, epsilon))

Omega_i = np.mean(Omega_i_values) # Expected value

# For Omega_O 
Omega_O_values = []
for epsilon in epsilon_to:  # With the real oponents stochastic values, we can't dondition opponent values
    Omega_O_values.append(optimizador_W(T, y, beta, a, b, epsilon))

Omega_O = np.mean(Omega_O_values) # Expected value

IndexError: invalid index to scalar variable.

________________

Estimacion de montecarlo no bayesiana

In [ ]:
def simulate_omega(W_function, epsilon_distributions, num_simulations=10000):
    """
    Simulate Omega value using Monte Carlo Simulation.
    :param W_function: The W function to maximize.
    :param epsilon_distributions: A function that generates realizations of epsilon.
    :param num_simulations: Number of simulations to run.
    :return: Estimated Omega value.
    """
    W_star_values = []

    for _ in range(num_simulations):
        epsilon_realization = epsilon_distributions()
        # Solve the maximization problem for W with the given epsilon
        result = maximize_W(W_function, epsilon_realization)
        if result.success:
            W_star_values.append(result.fun)  # Assuming maximization

    return np.mean(W_star_values)

# Example usage
# You need to define `maximize_W` function and `epsilon_distribution`
Omega_i = simulate_omega(W, epsilon_distribution_for_Omega_i)
Omega_O = simulate_omega(W, epsilon_distribution_for_Omega_O)


____________________

In [ ]:
# Parameters
beta = ...
X = ...
epsilon_i = ...

# Objective Function
def objective(vars, epsilon_ti, eta_ti):
    g, tau = vars
    pi_rho_value = pi_rho(tau, g, d, h, eta_ti)
    W_value = W(g, tau, epsilon_ti, y, a, b, beta)
    return -1 * (beta * (X * (1 + beta) + Omega_i - Omega_O) * pi_rho_value + W_value)


# Constraints
constraints = [
    {'type': 'ineq', 'fun': lambda vars: vars[0]},  # g >= 0
    {'type': 'ineq', 'fun': lambda vars: vars[1]},  # tau >= 0
    {'type': 'ineq', 'fun': lambda vars: vars[1] + epsilon_i - vars[0]}  # tau + epsilon_i - g >= 0
]

In [ ]:
# Initial guess
initial_guess = [1, 1]  # Adjust as needed

# Example time period t
t = ...  # specify the time period

# Solve the problem for time period t
result = minimize(lambda vars: objective(vars, epsilon_ti[t], eta_ti[t]), initial_guess, constraints=constraints)

# Output the result for time period t
if result.success:
    optimized_g, optimized_tau = result.x
    print(f"Optimized g: {optimized_g}, Optimized tau: {optimized_tau} for time period {t}")
else:
    print("Optimization failed for time period {t}.")